In [1]:
! pip install vininfo

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error


from category_encoders import TargetEncoder

import time

from vininfo import Vin

import warnings
warnings.filterwarnings('ignore')



Загрузим датасеты и произведем осмотр данных:

In [3]:
try:
    cars_train = pd.read_csv('C:/ya_pr/car_price_pred/train.csv')
    cars_test = pd.read_csv('C:/ya_pr/car_price_pred/test.csv')
except:
    print('Не смогла!')

In [4]:
cars_train.head(10)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)
5,2013,Lincoln,MKZ,Base,Sedan,automatic,3ln6l2j91dr817800,mi,2.5,14894.0,black,black,"ford motor credit company,llc",22600,Thu May 21 2015 02:00:00 GMT-0700 (PDT)
6,2010,pontiac,g6,4c,NaN,automatic,1g2za5eb4a4157380,nc,3.4,114587.0,silver,black,north state acceptance,5900,Mon Jan 12 2015 09:30:00 GMT-0800 (PST)
7,2013,Ford,Escape,SE,SUV,automatic,1fmcu0gx3duc59421,fl,4.8,26273.0,blue,gray,fields bmw,15200,Tue Feb 03 2015 01:00:00 GMT-0800 (PST)
8,2000,Hyundai,Elantra,GLS,Sedan,automatic,kmhjf35f2yu955691,oh,1.9,182624.0,black,tan,dt inventory,700,Thu Jan 22 2015 01:00:00 GMT-0800 (PST)
9,2005,Ford,Freestyle,Limited,wagon,automatic,1fmdk06135ga45438,oh,1.0,149364.0,black,tan,wells fargo dealer services,325,Tue Jun 16 2015 05:00:00 GMT-0700 (PDT)


In [5]:
cars_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [6]:
cars_train.loc[cars_train['make'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8043 entries, 10 to 440144
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          8043 non-null   int64  
 1   make          0 non-null      object 
 2   model         0 non-null      object 
 3   trim          0 non-null      object 
 4   body          0 non-null      object 
 5   transmission  6657 non-null   object 
 6   vin           8043 non-null   object 
 7   state         8043 non-null   object 
 8   condition     7977 non-null   float64
 9   odometer      8041 non-null   float64
 10  color         8029 non-null   object 
 11  interior      8029 non-null   object 
 12  seller        8043 non-null   object 
 13  sellingprice  8043 non-null   int64  
 14  saledate      8043 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory usage: 1005.4+ KB


In [7]:
cars_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  int64  
 1   make          107997 non-null  object 
 2   model         107979 non-null  object 
 3   trim          107944 non-null  object 
 4   body          107464 non-null  object 
 5   transmission  97047 non-null   object 
 6   vin           110058 non-null  object 
 7   state         110058 non-null  object 
 8   condition     107679 non-null  float64
 9   odometer      110039 non-null  float64
 10  color         109900 non-null  object 
 11  interior      109900 non-null  object 
 12  seller        110058 non-null  object 
 13  saledate      110058 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


In [8]:
cars_test.loc[cars_test['make'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2061 entries, 22 to 110016
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          2061 non-null   int64  
 1   make          0 non-null      object 
 2   model         0 non-null      object 
 3   trim          0 non-null      object 
 4   body          0 non-null      object 
 5   transmission  1707 non-null   object 
 6   vin           2061 non-null   object 
 7   state         2061 non-null   object 
 8   condition     2040 non-null   float64
 9   odometer      2060 non-null   float64
 10  color         2060 non-null   object 
 11  interior      2060 non-null   object 
 12  seller        2061 non-null   object 
 13  saledate      2061 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 241.5+ KB


 Общий осмотр позволяет сделать два вывода:
 1. Часть данных не имеет одновременно четырех важных признаков make, model, trim, body - это производитель, данные о модели и комплектации. Все признаки отвечающие за стоимость изначальную стоимость нового автомобиля. Такие данные в обучающем и тестовом наборах имеет смысл отделить и обучить на них свою отдельную модель. 
 2. Имеются и другие пропуски данных.
 3. Категориальные признаки включают дубли из-за написания слов с заглавной и строчной букв - имеет смысл привести все тексты к нижнему регистру:

In [9]:
categorial_columns = ['make','model','trim','body','transmission','vin','state','color','interior','seller']
numerical_columns = ['year','condition','odometer','sellingprice']

In [10]:
for column in categorial_columns:
    cars_train[column] = cars_train[column].str.lower()
    cars_test[column] = cars_test[column].str.lower()

In [11]:
cars_train[categorial_columns].describe()

,make,model,trim,body,transmission,vin,state,color,interior,seller
count,432193,432113,431899,429843,388775,440236,440236,439650,439650,440236
unique,62,839,1850,45,2,440236,38,20,17,13025
top,ford,altima,base,sedan,automatic,2fmdk3jc4bba41556,fl,black,black,nissan-infiniti lt
freq,74067,15454,43876,190861,375061,1,65618,87115,192442,15823


In [12]:
cars_train[numerical_columns].describe()

,year,condition,odometer,sellingprice
count,440236.000000,430831.000000,440167.000000,440236.000000
mean,2010.040101,3.425077,68344.421604,13592.209588
std,3.977945,0.949973,53542.203908,9751.479098
min,1982.000000,1.000000,1.000000,1.000000
25%,2007.000000,2.700000,28258.000000,6900.000000
50%,2012.000000,3.600000,52098.000000,12100.000000
75%,2013.000000,4.200000,99272.000000,18200.000000
max,2015.000000,5.000000,999999.000000,230000.000000


Обработаем пропуски (кроме данных относящихся к производителю и модели):

Трансмиссия. 

Закодируем значения automatic и manual как 1 и 0. Для строк где значение пропущено заполним средним по модели. Если после этого останутся пропущенные значения - заполним средним по датасету:

In [13]:
cars_train.loc[cars_train['transmission']=='automatic','transmission'] = 1
cars_train.loc[cars_train['transmission']=='manual','transmission'] = 0

cars_test.loc[cars_test['transmission']=='automatic','transmission'] = 1
cars_test.loc[cars_test['transmission']=='manual','transmission'] = 0

In [14]:
cars_train['transmission'] = cars_train['transmission'].fillna(cars_train.groupby('model')['transmission'].transform('mean'))
cars_test['transmission'] = cars_test['transmission'].fillna(cars_test.groupby('model')['transmission'].transform('mean'))

In [15]:
def transmission(transmission):
    if transmission >= 0:
        return transmission
    return 0.965

In [16]:
cars_train['transmission'] = cars_train['transmission'].apply(transmission)
cars_test['transmission'] = cars_test['transmission'].apply(transmission)

Из даты продажи выделим год, посчитаем возраст машины на момент продажи и удалим год производства (достаточно возраста и года продажи):

In [17]:
cars_train['sale_year'] = pd.to_datetime(cars_train.loc[0]['saledate'], utc=True).year
cars_train['age'] = cars_train['sale_year'] - cars_train['year']

In [18]:
cars_test['sale_year'] = pd.to_datetime(cars_test.loc[0]['saledate'], utc=True).year
cars_test['age'] = cars_test['sale_year'] - cars_test['year']

In [19]:
cars_train = cars_train.drop(['saledate','year'], axis=1)
cars_test = cars_test.drop(['saledate','year'], axis=1)

Пробег и состояние машины заполним соответствующими средними значениями по возрасту машины:

In [20]:
cars_train['odometer_by_age'] = cars_train.groupby('age')['odometer'].transform('mean')
cars_train['condition_by_age'] = cars_train.groupby('age')['condition'].transform('mean')

cars_test['odometer_by_age'] = cars_test.groupby('age')['odometer'].transform('mean')
cars_test['condition_by_age'] = cars_test.groupby('age')['condition'].transform('mean')

In [21]:
cars_train['condition'] = cars_train['condition'].fillna(cars_train['condition_by_age']) 
cars_train['odometer'] = cars_train['odometer'].fillna(cars_train['odometer_by_age']) 

cars_test['condition'] = cars_test['condition'].fillna(cars_test['condition_by_age']) 
cars_test['odometer'] = cars_test['odometer'].fillna(cars_test['odometer_by_age']) 

In [22]:
cars_train.loc[(cars_train['odometer']<100)&(cars_train['age']>0)
              ]['odometer'] = cars_train.loc[(cars_train['odometer']<100)&(cars_train['age']>0)]['odometer_by_age']

In [23]:
cars_test.loc[(cars_test['odometer']<100)&(cars_test['age']>0)
              ]['odometer'] = cars_test.loc[(cars_test['odometer']<100)&(cars_test['age']>0)]['odometer_by_age']

Vin удалим - он не влияет на стоимость машины.

In [24]:
cars_train = cars_train.drop('vin', axis=1)

In [25]:
categorial_columns = ['make','model','trim','body','state','color','interior','seller']
numerical_columns = ['condition','odometer','transmission','sale_year','sale_month','age','sellingprice']

Разделим данные на два набора под две разные модели (в соответствии с выводом №1 по итогам осмотра данных):

In [26]:
cars_train1 = cars_train.loc[cars_train['model'].isna()]
cars_train2 = cars_train.loc[~cars_train['model'].isna()]
print(cars_train1.shape)
print(cars_train2.shape)
cars_test1 = cars_test.loc[cars_test['model'].isna()]
cars_test2 = cars_test.loc[~cars_test['model'].isna()]
print(cars_test1.shape)
print(cars_test2.shape)

(8123, 16)
(432113, 16)
(2079, 16)
(107979, 16)


Проверки показали что для дорогих брендов имеет смысл так же выделить отдельную модель (отобраны по среденей цене продажи для каждого бренда):

In [27]:
cheap_makers = ['dodge','chrysler','hyundai','honda','buick','fiat','mazda','scion',
                'volkswagen','mitsubishi','mercedes','gmc truck','plymouth','mazda tk',
                'smart','suzuki','mercury','pontiac','saab','saturn','isuzu','oldsmobile',
                'daewoo','dodge tk','dot','ford truck','geo']

medium_makers = ['gmc','lincoln','subaru','hummer','cadillac','jeep','ford',
                 'acura','mercedes-b','mini','toyota','chevrolet','kia','nissan','vw','volvo','landrover']


lux_makers = ['rolls-royce','ferrari','lamborghini','bentley','airstream',
              'tesla','aston martin','fisker','maserati','porsche',
              'land rover','ram','mercedes-benz','bmw','infiniti','lexus','audi','jaguar']


In [28]:
#cars_train1 = cars_train.loc[cars_train['model'].isna()] 

#cars_train2 = cars_train.loc[~cars_train['model'].isna()]

cars_train3 = cars_train2.loc[cars_train2['make'].isin(lux_makers)] 

cars_train4 = cars_train2.loc[~cars_train2['make'].isin(lux_makers)]




#print(cars_train1.shape)
#print(cars_train2.shape)
print(cars_train3.shape)
print(cars_train4.shape)

print()
cars_test1 = cars_test.loc[cars_test['model'].isna()]

#cars_test2 = cars_test.loc[~cars_test['model'].isna()]

cars_test3 = cars_test2.loc[cars_test2['make'].isin(lux_makers)]

cars_test4 = cars_test2.loc[~cars_test2['make'].isin(lux_makers)]



#print(cars_test1.shape)
#print(cars_test2.shape)
print(cars_test3.shape)
print(cars_test4.shape)






(63169, 16)
(368944, 16)

(16025, 16)
(91954, 16)


In [29]:
train_1, valid_1 = train_test_split(cars_train1, test_size=0.1, random_state=321)
#train_2, valid_2 = train_test_split(cars_train2, test_size=0.1, random_state=321)
train_3, valid_3 = train_test_split(cars_train3, test_size=0.1, random_state=321)
train_4, valid_4 = train_test_split(cars_train4, test_size=0.1, random_state=321)


In [30]:
test_1 = cars_test1.copy()
#test_2 = cars_test2.copy()
test_3 = cars_test3.copy()
test_4 = cars_test4.copy()


In [31]:
target_encoder = TargetEncoder()
target_encoder.fit(cars_train[categorial_columns],cars_train['sellingprice'])

train_1[categorial_columns] = target_encoder.transform(train_1[categorial_columns])
valid_1[categorial_columns] = target_encoder.transform(valid_1[categorial_columns])

test_1[categorial_columns] = target_encoder.transform(test_1[categorial_columns])

#train_2[categorial_columns] = target_encoder.transform(train_2[categorial_columns])
#valid_2[categorial_columns] = target_encoder.transform(valid_2[categorial_columns])

#test_2[categorial_columns] = target_encoder.transform(test_2[categorial_columns])

train_3[categorial_columns] = target_encoder.transform(train_3[categorial_columns])
valid_3[categorial_columns] = target_encoder.transform(valid_3[categorial_columns])

test_3[categorial_columns] = target_encoder.transform(test_3[categorial_columns])

train_4[categorial_columns] = target_encoder.transform(train_4[categorial_columns])
valid_4[categorial_columns] = target_encoder.transform(valid_4[categorial_columns])

test_4[categorial_columns] = target_encoder.transform(test_4[categorial_columns])

In [32]:
X_train_1 = train_1.drop('sellingprice', axis=1)
y_train_1 = train_1['sellingprice']

X_valid_1 = valid_1.drop('sellingprice', axis=1)
y_valid_1 = valid_1['sellingprice']

In [33]:
#X_train_2 = train_2.drop('sellingprice', axis=1)
#y_train_2 = train_2['sellingprice']

#X_valid_2 = valid_2.drop('sellingprice', axis=1)
#y_valid_2 = valid_2['sellingprice']

In [34]:
X_train_3 = train_3.drop('sellingprice', axis=1)
y_train_3 = train_3['sellingprice']

X_valid_3 = valid_3.drop('sellingprice', axis=1)
y_valid_3 = valid_3['sellingprice']

In [35]:
X_train_4 = train_4.drop('sellingprice', axis=1)
y_train_4 = train_4['sellingprice']

X_valid_4 = valid_4.drop('sellingprice', axis=1)
y_valid_4 = valid_4['sellingprice']

In [36]:
def test_model(model, features_train, features_test, target_train, target_test):
    
    fit_start_time = time.time()
    model.fit(features_train, target_train)
    fit_end_time = time.time()
    
    predict_start_time = time.time()
    predict = model.predict(features_test)
    predict_end_time = time.time()
    
    print('Время на обучение:', fit_end_time - fit_start_time)
    print('Время на прогноз:', predict_end_time - predict_start_time)
    print('MAPE на валидационной выборке:', mean_absolute_percentage_error(target_test, predict))

In [37]:
rate = 0.5
model_1 = CatBoostRegressor(verbose=False, learning_rate=rate)
test_model(model_1, X_train_1, X_valid_1, y_train_1, y_valid_1)
print('learning_rate=',rate)

Время на обучение: 2.993382215499878
Время на прогноз: 0.0052068233489990234
MAPE на валидационной выборке: 0.6867124185360741
learning_rate= 0.5


In [38]:
#rate = 0.5
#model_2 = CatBoostRegressor(verbose=False, learning_rate=rate)
#test_model(model_2, X_train_2, X_valid_2, y_train_2, y_valid_2)
#print('learning_rate=',rate)

In [39]:
rate = 0.5
model_3 = CatBoostRegressor(verbose=False, learning_rate=rate)
test_model(model_3, X_train_3, X_valid_3, y_train_3, y_valid_3)
print('learning_rate=',rate)

Время на обучение: 7.650482177734375
Время на прогноз: 0.004996299743652344
MAPE на валидационной выборке: 0.10324053974402357
learning_rate= 0.5


In [40]:
rate = 0.5
model_4 = CatBoostRegressor(verbose=False, learning_rate=rate)
test_model(model_4, X_train_4, X_valid_4, y_train_4, y_valid_4)
print('learning_rate=',rate)

Время на обучение: 36.72625708580017
Время на прогноз: 0.012990474700927734
MAPE на валидационной выборке: 0.15006971940421757
learning_rate= 0.5


In [41]:
vin_test_1 = test_1['vin']
X_test_1 = test_1.drop('vin', axis=1)

#vin_test_2 = test_2['vin']
#X_test_2 = test_2.drop('vin', axis=1)

vin_test_3 = test_3['vin']
X_test_3 = test_3.drop('vin', axis=1)

vin_test_4 = test_4['vin']
X_test_4 = test_4.drop('vin', axis=1)
 



In [42]:
predict_1 = model_1.predict(X_test_1)
#predict_2 = model_2.predict(X_test_2)
predict_3 = model_3.predict(X_test_3)
predict_4 = model_4.predict(X_test_4)

In [43]:
sub_1 = pd.DataFrame()
sub_1['vin'] = vin_test_1
sub_1['sellingprice'] = predict_1
sub_1.loc[sub_1['sellingprice']<=400,'sellingprice'] = 400
#sub_2 = pd.DataFrame()
#sub_2['vin'] = vin_test_2
#sub_2['sellingprice'] = predict_2

sub_3 = pd.DataFrame()
sub_3['vin'] = vin_test_3
sub_3['sellingprice'] = predict_3
sub_3.loc[sub_3['sellingprice']<=400,'sellingprice'] = 400

sub_4 = pd.DataFrame()
sub_4['vin'] = vin_test_4
sub_4['sellingprice'] = predict_4
sub_4.loc[sub_4['sellingprice']<=400,'sellingprice'] = 400

In [44]:
submission = pd.concat([sub_1,sub_3,sub_4])

In [45]:
submission.to_csv('C:/ya_pr/car_price_pred/submission2.csv', sep=',', index=False)